In [ ]:
from litellm import completion
from typing import List, Dict, Any, Optional
import jinja2
import json
from abc import ABC, abstractmethod
from tavily import TavilyClient
import os




# Objective: Create a conversation object that can be used to send messages to the model and get responses and tools
class Conversation:
    def __init__(self, model: str = "gemini/gemini-2.0-flash"):
        self.model = model
        self.messages = []

    def add_message(self, role, content):
        self.messages.append({"role": role, "content": content})

    def get_response(self, tool_choice: str = "auto", tools: Optional[List] = None):
        if tools is None:
            tool_choice = None
        return completion(
            model=self.model,
            messages=self.messages,
            tools=tools,
            tool_choice=tool_choice,
        )


class Tool(ABC):
    def __init__(self):
        pass

    # method to run the tool.
    @abstractmethod
    def run(self, conversation: Conversation) -> str:
        pass

    # method to get config for the tool.
    @abstractmethod
    def get_config(self) -> Dict[str, Any]:
        pass


class Reasoning(Tool):
    def __init__(self):
        self.name = "reasoning"

    def run(self, conversation: Conversation) -> str:
        with open("templates/reasoning.jinja2", "r") as file:
            reasoning_template = jinja2.Template(file.read()).render()
        conversation.add_message("system", reasoning_template)
        response = conversation.get_response()
        conversation.messages.pop()  # remove the reasoning message and note that no other message has been added to the conversation as add_message method from Conversation class has not been called.
        return response.choices[0].message.content

    def get_config(self) -> Dict[str, Any]:
        return {
            "type": "function",
            "function": {
                "name": self.name,
                "description": "This tool provides reasoning for the given context.",
            },
        }

'''Repond tool can now decide the type of response to be generated based on the response_type parameter passed to it.'''
class Respond(Tool):
    def __init__(self, response_types: List[str]):
        self.name = "respond"
        self.response_types = response_types

    def run(self, conversation: Conversation, response_type: str) -> str:
        with open(f"templates/{response_type}.jinja2", "r") as file:
            reasoning_template = jinja2.Template(file.read()).render()
        conversation.add_message("system", reasoning_template)
        response = conversation.get_response()
        conversation.messages.pop()
        return response.choices[0].message.content

    def get_config(self) -> dict:
        return {
            "type": "function",
            "function": {
                "name": self.name,
                "description": "This tool provides responses for the given user message.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "response_type": {
                            "type": "string",
                            "description": "The type of response to be generated.",
                            "enum": self.response_types,
                        },
                    },
                    "required": ["response_type"],
                },
            },
        }


# An agent is a tool that can be used to interact with the model and get responses and use tools as required.
class Agent(Tool):
    def __init__(
        self,
        name: str,
        description: str,
        max_turns: int = 3,
        tools: List[Tool] = [],
        terminating_tool: Tool = None,
    ):
        self.name = name
        self.description = description
        self.max_turns = max_turns
        self.available_tools = {tool.name: tool for tool in tools}
        self.terminating_tool = terminating_tool

    def run(self, conversation: Conversation) -> str:
        for turn in range(self.max_turns):
            tool, function_args = self.get_next_action(conversation)
            print(f"Using the {tool.name} tool with arguments {function_args}")
            response = tool.run(conversation, **function_args)
            if tool.name == self.terminating_tool.name:
                return response

            conversation.add_message(
                "user",
                f"The {tool.name} tool has been used and it responded with {response}",
            )
        return self.terminating_tool.run(self.conversation)

    def get_next_action(self, conversation: Conversation) -> Tool:
        response = conversation.get_response(
            tool_choice="required",
            tools=[tool.get_config() for tool in self.available_tools.values()],
        )
        tool_calls = response.choices[0].message.tool_calls
        function_name = tool_calls[0].function.name
        function_args = json.loads(tool_calls[0].function.arguments)
        return self.available_tools[function_name], function_args

    def get_config(self) -> dict:
        return {
            "type": "function",
            "function": {
                "name": self.name,
                "description": self.description,
            },
        }
    
# Search Tool
class Search(Tool):
    def __init__(self):
        self.name = "search"
        self.tavily_client = TavilyClient(api_key=os.getenv("TAVILY_API_KEY"))

    def run(self, conversation: Conversation, query: str) -> str:
        response = self.tavily_client.search(query)
        return response
    
    def get_config(self) -> dict:
        return {
            "type": "function",
            "function": {
                "name": self.name,
                "description": "This tool provides search results for the given query.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "query": {
                            "type": "string",
                            "description": "The query to search for.",
                        },
                    },
                    "required": ["query"],
                },
            },
        }



Testing

In [14]:
with open("templates/system_message.jinja2", "r") as file:
    system_message = jinja2.Template(file.read()).render(tools="")

In [17]:
write_report = Respond(["write_report"])

research_agent = Agent(
    name="research_agent",
    description="This agent can be used to research on a given topic.",
    max_turns=10,
    tools=[Reasoning(), write_report, Search()],
    terminating_tool=write_report,
)

blarg = Conversation()

blarg.add_message("user", "I want to research on the topic of AI.")

print(research_agent.run(blarg))

Using the search tool with arguments {'query': 'AI'}
Using the reasoning tool with arguments {}
Using the respond tool with arguments {'response_type': 'write_report'}


"Okay, here's a comprehensive report based on the provided research data and the outlined plan.  Since the initial data is limited, the report will provide a foundational overview of AI and suggest areas for further investigation.\n\n**Artificial Intelligence: An Overview**\n\n**1. Definition and Core Concepts:**\n\nArtificial Intelligence (AI) is a broad field focused on creating machines capable of performing tasks that typically require human intelligence.  These tasks include:\n\n*   **Reasoning:**  The ability to draw inferences and solve problems.\n*   **Knowledge Representation:**  Storing and organizing information in a way that a machine can use.\n*   **Planning:**  Devising sequences of actions to achieve goals.\n*   **Learning:**  Improving performance based on experience (data).\n*   **Natural Language Processing (NLP):**  Understanding and generating human language.\n*   **Perception:**  Interpreting sensory data (e.g., images, sound).\n*   **Robotics:** Designing and buil

In [18]:


conversation = Conversation()
conversation.add_message("system", system_message)

respond = Respond(response_types=["respond", "clarification"])

agent = Agent(
    name="Autonomous market research agent",
    description="This agent is designed to help users with market research.",
    tools=[
        Reasoning(),
        respond,
        research_agent
    ],
    terminating_tool=respond,
)

conversation.add_message("user", "I want to research climate litigation")


response = agent.run(conversation)

print(response)

Using the research_agent tool with arguments {}
Using the search tool with arguments {'query': 'climate litigation'}
Using the reasoning tool with arguments {}
Using the respond tool with arguments {'response_type': 'write_report'}
Using the respond tool with arguments {'response_type': 'clarification'}
Okay, I see that you're interested in researching climate litigation. Based on your initial query and the search results, it seems like you want to understand:

*   **What climate litigation is:** The definition, types, and trends of climate change litigation.
*   **Key players:** Who is involved in these cases (plaintiffs and defendants).
*   **Goals of climate litigation:** What these cases aim to achieve (policy changes, compensation, etc.).
*   **Impact and challenges:** The successes, limitations, and overall impact of climate litigation.

Is that a correct assessment of what you're hoping to achieve with this research? Are there any specific aspects of climate litigation that you'